In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import pydicom
import cv2
import os
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import _pickle as pickle

def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

In [ ]:
imSize = 128

def find_bound(im, tres, axis):
    inf = 0
    sup = im.shape[axis]
    cond = True
    tres = tres
    temp = 0
    for i,elt in enumerate(im.sum(axis = axis)):
        if abs(elt) <=tres and cond == True:
            temp = i
        if abs(elt) >=tres:
            cond = False
            inf = temp
            

        if abs(elt)  <= tres and cond == False:
            sup = i
            cond = True
    return inf, sup

def preprocess_image(image_path, desired_size=imSize):
#     print(image_path)
    im = pydicom.dcmread(image_path).pixel_array
    
#     im = (im - im.min())/(im.max()-im.min())
    tres = 5
    inf_x, sup_x = find_bound(im, tres, 1)
    inf_y, sup_y = find_bound(im,tres,0)
    im = im[inf_x:sup_x,inf_y: sup_y]
    im = cv2.resize(im, (imSize,imSize))
    return im

def build_3d_image(ids, res = 256, to_keep = 'ALL'):
    path = './train/' + ids

    list_dir = [int(elt.split('.')[0]) for elt in os.listdir(path)]
    list_dir.sort()
    
    if to_keep == 'ALL':
        n_image_keep = len(list_dir)
    else:
        n_image_keep = to_keep
    X = list(np.zeros(n_image_keep))  #list(np.zeros(len(list_dir)))

    n_img = len(list_dir)
    print(n_img)

    for i in range(n_image_keep):
        elt = list_dir[int(i*n_img/(n_image_keep))]
        im = preprocess_image(path + '/'+ str(elt) + '.dcm', desired_size = res)
        X[i] = im
#         print(int(i*n_img/(n_image_keep)))

    X = np.array(X).astype(float)
    
    X = X.transpose(1, 2, 0)
    # X = cv2.resize(X, (imSize,imSize))
    
    X = (X - X.min())/(X.max()-X.min())
    
    return X

def make_3d_vid(X):
    %matplotlib notebook
    fig1 = plt.figure(num='Lung', figsize = (5,5))

    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel('x label')

    end = X.shape[0]
    for i in range(end):
        ax1.cla()  # Clear only 2nd figure's axes, figure 1 is ADDITIVE
        ax1.set_title('Axes title')  # Reset as removed by cla()

        ax1.imshow(X[i,:,:], cmap = 'gray')
    #     ax2.plot(range(i,end), range(i,end), 'rx')
        fig1.canvas.draw()
    #     plt.pause(0.001)

    %matplotlib notebook
    fig1 = plt.figure(num='Lung', figsize = (5,5))

    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel('x label')

    end = X.shape[1]
    for i in range(end):
        ax1.cla()  # Clear only 2nd figure's axes, figure 1 is ADDITIVE
        ax1.set_title('Axes title')  # Reset as removed by cla()

        ax1.imshow(X[:,i,:], cmap = 'gray')
    #     ax2.plot(range(i,end), range(i,end), 'rx')
        fig1.canvas.draw()
    #     plt.pause(0.001)

    %matplotlib notebook
    fig1 = plt.figure(num='Lung', figsize = (5,5))

    ax1 = fig1.add_subplot(111)
    ax1.set_xlabel('x label')

    end = X.shape[2]
    for i in range(end):
        ax1.cla()  # Clear only 2nd figure's axes, figure 1 is ADDITIVE
        ax1.set_title('Axes title')  # Reset as removed by cla()

        ax1.imshow(X[:,:,i], cmap = 'gray')
    #     ax2.plot(range(i,end), range(i,end), 'rx')
        fig1.canvas.draw()
    #     plt.pause(0.001)

In [ ]:
ids = 'ID00014637202177757139317'
ind = 4
ids = os.listdir('./train')[ind]
ids = 'ID00232637202260377586117'


X = build_3d_image(ids,res = 256, to_keep = 32)

# make_3d_vid(X)

In [ ]:
df = pd.read_csv('train.csv')
df = df.sort_values(by = ['Weeks'])
df.head()

ids  =[]
data = []
FVC0 = []
FVC1 = []
STD = []

for i,elt in enumerate(tqdm(os.listdir('./train'))):
    
    id1 = elt
    df1 = df[df['Patient'] == elt]
    m = 100
    ind = 0
    for i, elt1 in enumerate(df1['Weeks']):
        if abs(elt1)<m:
            m = abs(elt1)
            ind = i
#     FVC0.append(df1.iloc[ind]['FVC'])
#     FVC1.append(df1.iloc[-1]['FVC'])
#     STD.append(df1['FVC'].values.std())
    
    for img in os.listdir('./train/'+elt):
        try:
            image_path = './train/'+elt+'/'+img
            imSize = imSize
            im = preprocess_image(image_path, desired_size=imSize)
            im = np.array(im).astype(float)
            im = (im - im.min())/(im.max()-im.min())
            if str(im.sum())!='nan':
                ids.append(id1)
                data.append(im)
                FVC0.append(df1.iloc[ind]['FVC'])
                FVC1.append(df1.iloc[-1]['FVC'])
                STD.append(df1['FVC'].values.std())
        except:
            print('miss')

ids = np.array(ids)
data = np.array(data)
FVC0 = np.array(FVC0)
FVC1 = np.array(FVC1)
STD = np.array(STD)

In [ ]:
data.shape

In [ ]:
save((ids, data, FVC0, FVC1, STD), 'dataset_2D')


## DEEP LEARNING

In [ ]:
from tensorflow.keras.layers import Conv3DTranspose, Activation,Conv2DTranspose, BatchNormalization, Flatten, LeakyReLU, MaxPool3D,Conv2D, Conv3D, AveragePooling3D, Reshape, Input, Dense, Add
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD,Adam
import tensorflow.keras.backend as K

In [ ]:
(ids, data, FVC0, FVC1, STD) = load('dataset_2D')



In [ ]:
FVC0 = np.array(FVC0).astype(float)/1000
FVC1 = np.array(FVC1).astype(float)/1000
STD = np.array(STD).astype(float)/100

In [ ]:
ids.shape

In [ ]:
for i, elt in enumerate(data):
    if str(elt.sum()) == 'nan':
        print(i)

In [ ]:
data = data.reshape((data.shape[0], data.shape[1],data.shape[2],1))
X_train, X_test, fvc0_train, fvc0_test = train_test_split(data, FVC0, test_size=0.1, random_state=42)
fvc1_train, fvc1_test, std_train, std_test = train_test_split(FVC1, STD, test_size=0.1, random_state=42)

y_train = [fvc0_train, fvc1_train, std_train]
y_test = [fvc0_test, fvc1_test, std_test]

del data
import gc
gc.collect()

In [ ]:
def build_encoder(input_shape, latent_dim = 512):
    inputs = Input(shape = input_shape)
    
    x = Conv2D(32, (7,7), strides = 1, padding = 'same')(inputs)
    
    filters = [32,32,64,64,128]
    for f in filters:
        x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = BatchNormalization(axis=-1)(x)
    volumeSize = K.int_shape(x)   
    x = Flatten()(x)
    
    outputs = Dense(latent_dim)(x)
    
    model = Model(inputs, outputs)
    return model, volumeSize

def build_decoder(volumeSize, latent_dim = 512):
    
#     inputs = Input(shape = (512,))
    
    inputs = Input(shape=(latent_dim,))
    x = Dense(np.prod(volumeSize[1:]))(inputs)
    x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
    # loop over our number of filters again, but this time in
    # reverse order
    filters = [128,64,64,32,32]
    for f in filters[::-1]:
        # apply a CONV_TRANSPOSE => RELU => BN operation
        x = Conv2DTranspose(f, (3, 3), strides=2,
                        padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = BatchNormalization(axis=-1)(x)
    
    x = Conv2D(1, (3, 3), padding="same")(x)
    outputs = Activation("sigmoid")(x)
    
    model = Model(inputs, outputs)
    return model

In [ ]:
input_shape = (imSize,imSize,1)
encoder, shape = build_encoder(input_shape, 512)
decoder = build_decoder(shape, 512)

In [ ]:
encoder.summary()
decoder.summary()

In [ ]:
input_shape = (imSize,imSize,1)
encoder, shape = build_encoder(input_shape, 2048)
decoder = build_decoder(shape, 2048)

inputs = Input(shape = input_shape)

encoded = encoder(inputs)
decoded = decoder(encoded)

model = Model(inputs, decoded)

In [ ]:

loss = 'mse'
optimizer = SGD(0.1)
optimizer = Adam(lr=1e-3)
metrics = []

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

In [ ]:
import tensorflow
stop = tensorflow.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.0001, patience=6, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)
reduce = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                                                     mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)


batch_size = 64
epochs = 90

history = model.fit(X_train, X_train, batch_size=batch_size,
    validation_data=(X_test, X_test),  epochs=epochs, callbacks = [stop, reduce])

In [ ]:
X_pred = model.predict(X_test)

In [ ]:
ind = 114


size = imSize
plt.figure(1)
plt.imshow(X_test[ind].reshape((size,size)), cmap = 'gray')

plt.figure(2)
plt.imshow(X_pred[ind].reshape((size,size)), cmap = 'gray')

In [ ]:
encoder.save_weights('./checkpoints/encoder/encoder')
decoder.save_weights('./checkpoints/decoder/decoder')

In [ ]:
a=encoder.predict(X_test)

In [ ]:
a.shape

In [ ]:
FVC0.mean()

In [ ]:
FVC1.mean()

In [ ]:
save((ids, data, FVC0, FVC1, STD), 'dataset')